<h1 style="font-size: 50px;">Research Project 2 Epilepsy Ionic Modulation - SmallBrain Old Code</h1>

PLACEHOLDER FOR SMALL EXPLANATION

<br></br><br></br>

<h2 style="font-size: 40px;">SmallBrain Hippocampus Layered.ipynb</h2>

In [12]:
# Calculating the middle y-coordinate of the stimulus as well as its radius.
def calculate_spherical_stimulus_middle_y_point_and_radius(layer_data, list_of_layers):

    # Initializing a cumulative y-coordinate that represents the height of all the layers.
    current_total_y = 0

    # Initializing a cumulative y-coordinate that represents the height of the layers only in the 'list_of_layers'.
    current_layers_y = 0

    # Initializing the variables 'y_middle_point' and 'radius_of_stimulus' that will eventually respectively contain the middle y-coordinate of the current layer as well as a radius that makes sure only the neurons from the current layer are part of the stimulus mask.
    y_middle_point = 0
    radius_of_stimulus = 0

    # Looping over every layer and if the current layer is the layer that we are looking for, the 'current_y' variable can be used to retrieve the middle y-coordinate of the current layer. Else, add the layer height to the 'current_y' variable.
    for layer in layer_data:
        if len(list_of_layers) == 1: 
            layer_name = list_of_layers[0]
            if layer_data[layer]['layer_name'] == layer_name:
                y_middle_point = current_total_y + (layer_data[layer]['height'] / 2)
                radius_of_stimulus = (layer_data[layer]['height'] / 2) * 0.99  # This is multiplied by 0.99 to ensure that we stay within the bounds of the current layer.
            else:
                current_total_y += layer_data[layer]['height']
        
        else:
            if layer_data[layer]['layer_name'] in list_of_layers:
                if layer_data[layer]['layer_name'] == list_of_layers[-1]:
                    y_middle_point = current_total_y + ((current_layers_y + layer_data[layer]['height']) / 2)
                    radius_of_stimulus = ((current_layers_y + layer_data[layer]['height']) / 2) * 0.99  # This is multiplied by 0.99 to ensure that we stay within the bounds of the set of layers.
                else:
                    current_layers_y += layer_data[layer]['height']
            else:
                current_total_y += layer_data[layer]['height']

    return [y_middle_point, radius_of_stimulus]

In [14]:
import random

# Creating the complete neuron topology of the model taking into account the layered structure of the hippocampus.
def create_complete_neuron_topology_hippocampus_layered(total_num_of_neurons, random_topo_bounds, layer_neuron_densities, layer_excitatory_ratios, layer_names):
    
    # Calculating the neuron density of the whole random topology model.
    random_x_bound, random_y_bound, random_z_bound = random_topo_bounds 
    random_topo_volume = random_x_bound * random_y_bound * random_z_bound
    random_topo_density = total_num_of_neurons / random_topo_volume

    # Calculating the sum of desired neuron densities across all the hippocampal layers.
    total_desired_density = sum(list(layer_neuron_densities.values()))

    # Calculating the total number of neurons for each layer based on the proportion of their density to the total desired density.
    neuron_counts = {layer: (density / total_desired_density) * total_num_of_neurons for layer, density in layer_neuron_densities.items()}
    
    # Initializing a dictionary to store information about the layers.
    layer_data = {}

    # Looping over each layer and calculating its volume and neuron count.
    for layer_name in layer_names:
        layer_density = layer_neuron_densities[layer_name]
        
        # Total neurons in the current layer
        layer_total_neurons = neuron_counts[layer_name]

        # Calculating the volume of the current layer.
        layer_volume = layer_total_neurons / random_topo_density

        # Calculating the total number of excitatory and inhibitory neurons in the current layer.
        layer_num_excitatory_neurons = layer_excitatory_ratios[layer_name] * layer_total_neurons
        layer_num_inhibitory_neurons = (1 - layer_excitatory_ratios[layer_name]) * layer_total_neurons

        # Fixing the width and the depth of the neuron layer such that only the height of the layers varies.
        layer_width = random_x_bound
        layer_depth = random_z_bound

        # Calculating the height of the layer based on the volume and the width and depth of the layer.
        layer_height = layer_volume / (layer_width * layer_depth)

        # Adding the layer data to the 'layer_data' dictionary.
        layer_data[layer_name] = {
            "layer_name": layer_name,
            "volume": layer_volume,
            "width": layer_width,
            "depth": layer_depth,
            "height": layer_height,
            "num_excitatory_neurons": int(layer_num_excitatory_neurons),
            "num_inhibitory_neurons": int(layer_num_inhibitory_neurons)         
        }

    # Initializing a cumulative y-coordinate that will be used for generating the different points across the layers.
    current_y = 0

    # Initializing two dictionaries that will for each layer store the positions of the excitatory and inhibitory neurons.
    excitatory_positions = {}
    inhibitory_positions = {}

    # Looping over every layer and randomly generating a set of excitatory and inhibitory neurons according to the neurons defined in the 'layer_data'.
    for layer in layer_data:
        layer_all_info = layer_data[layer]

        # Retrieving the number of excitatory and inhibitory neurons that should be present within the current layer.
        num_excitatory_neurons = layer_all_info['num_excitatory_neurons']
        num_inhibitory_neurons = layer_all_info['num_inhibitory_neurons']

        # Generating a set of random x, y, and z positions that fall within the current layer for the excitatory neurons.
        layer_x_positions_exc = [random.uniform(0, layer_all_info['width']) for _ in range(num_excitatory_neurons)]
        layer_y_positions_exc = [(current_y + random.uniform(0, layer_all_info['height'])) for _ in range(num_excitatory_neurons)]
        layer_z_positions_exc = [random.uniform(0, layer_all_info['depth']) for _ in range(num_excitatory_neurons)]
        excitatory_topology_layer = np.array([layer_x_positions_exc, layer_y_positions_exc, layer_z_positions_exc])

        # Generating a set of random x, y, and z positions that fall within the current layer for the inhibitory neurons.
        layer_x_positions_inh = [random.uniform(0, layer_all_info['width']) for _ in range(num_inhibitory_neurons)]
        layer_y_positions_inh = [(current_y + random.uniform(0, layer_all_info['height'])) for _ in range(num_inhibitory_neurons)]
        layer_z_positions_inh = [random.uniform(0, layer_all_info['depth']) for _ in range(num_inhibitory_neurons)]
        inhibitory_topology_layer = np.array([layer_x_positions_inh, layer_y_positions_inh, layer_z_positions_inh])

        # Updating the cumulative y-coordinate.
        current_y += layer_all_info['height']

        # Adding the 'excitatory_topology_layer' and the 'inhibitory_topology_layer' to the dictionaries 'excitatory_positions' and 'inhibitory_positions'.
        excitatory_positions[layer_all_info['layer_name']] = excitatory_topology_layer
        inhibitory_positions[layer_all_info['layer_name']] = inhibitory_topology_layer
        
    return [layer_data, excitatory_positions, inhibitory_positions]

####################################
########  TESTING FUNCTION  ########
####################################
total_num_of_neurons = 17000
random_topo_bounds = [0.6, 0.6, 0.6]
layer_neuron_densities = {"EC": 27400, 
                          "DG": 87100, 
                          "CA1": 7900, 
                          "CA3": 15100, 
                          "Sub": 21500}
layer_excitatory_ratios = {"EC": 0.8, 
                           "DG": 0.8, 
                           "CA1": 0.8, 
                           "CA3": 0.8, 
                           "Sub": 0.8}
layer_names = ['EC', 'DG', 'CA1', 'CA3', 'Sub']

layer_data, excitatory_positions, inhibitory_positions = create_complete_neuron_topology_hippocampus_layered(total_num_of_neurons, random_topo_bounds, layer_neuron_densities, layer_excitatory_ratios, layer_names)

# Calculating the total number of neurons and the total density for verification.
total_num_of_neurons_ver = 0
total_volume = 0
for layer in layer_names:
    total_num_of_neurons_ver += (len(excitatory_positions[layer][0]) + len(inhibitory_positions[layer][0]))
    total_volume += layer_data[layer]['volume']

print(f"Total DESIRED number of neurons: {total_num_of_neurons}")
print(f"Total ACTUAL number of neurons: {total_num_of_neurons_ver}")
print(f"Total DESIRED neuron density: {total_num_of_neurons / (random_topo_bounds[0] * random_topo_bounds[1] * random_topo_bounds[2])}")
print(f"Total ACTUAL neuron density: {total_num_of_neurons_ver / total_volume}")
print(layer_data)
print(excitatory_positions)

Total DESIRED number of neurons: 17000
Total ACTUAL number of neurons: 16993
Total DESIRED neuron density: 78703.70370370371
Total ACTUAL neuron density: 78671.2962962963
{'EC': {'layer_name': 'EC', 'volume': 0.037222641509433954, 'width': 0.6, 'depth': 0.6, 'height': 0.10339622641509431, 'num_excitatory_neurons': 2343, 'num_inhibitory_neurons': 585}, 'DG': {'layer_name': 'DG', 'volume': 0.11832452830188678, 'width': 0.6, 'depth': 0.6, 'height': 0.32867924528301884, 'num_excitatory_neurons': 7450, 'num_inhibitory_neurons': 1862}, 'CA1': {'layer_name': 'CA1', 'volume': 0.010732075471698114, 'width': 0.6, 'depth': 0.6, 'height': 0.029811320754716982, 'num_excitatory_neurons': 675, 'num_inhibitory_neurons': 168}, 'CA3': {'layer_name': 'CA3', 'volume': 0.02051320754716981, 'width': 0.6, 'depth': 0.6, 'height': 0.0569811320754717, 'num_excitatory_neurons': 1291, 'num_inhibitory_neurons': 322}, 'Sub': {'layer_name': 'Sub', 'volume': 0.029207547169811318, 'width': 0.6, 'depth': 0.6, 'height':

In [14]:

# Also change this such that we put the stimulus mask in other places in a singular layer (or multiple layers) as well. Use some kind of scaling where 1 is on the edge and 0 in the center (do this for all three dimensions).

def create_stimulus_mask(shape_mask, list_of_layers, layer_names):

    # Checking if the 'list_of_layers' is empty.
    if len(list_of_layers) == 0:
        raise ValueError("The 'list_of_layers' list is empty.")
    
    # Checking if the 'list_of_layers' only contains valid layer names.
    if not all(layer in layer_names for layer in list_of_layers):
        raise ValueError("The 'list_of_layers' list contains an invalid layer name.")
    
    # Checking if there are no duplicate layer in the 'list_of_layers'.
    if len(list_of_layers) != len(set(list_of_layers)):
        raise ValueError("The 'list_of_layers' list contains duplicate layers.")

    # Checking that the 'list_of_layers' list only contains layers that are adjacent to each other (otherwise the stimulus mask would pass through layers that are not in this list).
    if len(list_of_layers) > 1:
        layer_indices = [layer_names.index(layer) for layer in list_of_layers]
        if not all(layer_indices[i] + 1 == layer_indices[i+1] for i in range(len(layer_indices) - 1)):
            raise ValueError("The 'list_of_layers' list contains layers that are not adjacent to each other.")

    
    if shape_mask == "spherical":
        # Setting the middle point of the stimulus mask.
        stimulus_layer = list_of_layers[0]
        x_middle_point = layer_data[stimulus_layer]['width'] / 2
        z_middle_point = layer_data[stimulus_layer]['depth'] / 2
        y_middle_point, radius_of_stimulus = calculate_stimulus_middle_y_point_and_radius(layer_data, list_of_layers)
        coord_of_stimulus = [[x_middle_point, y_middle_point, z_middle_point]]
    
        return [coord_of_stimulus, radius_of_stimulus]


    elif shape_mask == "cubical":
        if len(list_of_layers) == 1:


        else:
            

    
    else:
        raise ValueError("The shape of the mask given is not valid. Please select either 'spherical' or 'cubical'.")


In [16]:
layer_names = ['EC', 'DG', 'CA1', 'CA3', 'Sub']
create_stimulus_mask('spherical', ['EC'], layer_names)

In [11]:
# This function manages the overall process of setting up and running multiple instances of a neural network simulation as after setting everything up, the function 'run_granular_simulation()' is called.
# In addition, it also handles the storing of the results.
def run_model_loop(variables):

    # Checking whether lengths of each variable list is equal.
    if not check_dict_lenghts(variables):
        raise ValueError('Lenghts of each variable list has be to equal!')

    
    ############################################
    ########  Setting up Stimulus Mask  ########
    ############################################
    # Setting which layer(s) will feature the stimulus mask and what kind of stimulus mask should be constructed.
    stimulus_layers = ['EC']
    shape_mask = 'spherical'

    # Creating the stimulus mask by calling the function 'create_stimulus_mask()'.
    coord_of_stimulus, radius_of_stimulus = create_stimulus_mask(shape_mask, stimulus_layers, variables['layer_names'])
    
    # Setting the overall geometry settings of the stimulus mask which includes both the middle point as well as the radius.
    stimulus_geometry_settings = [coord_of_stimulus, radius_of_stimulus]

    # Updating the 'variables'.
    variables['coord_of_stimulus'] = coord_of_stimulus * variables['copy_times']
    variables['radius_of_stimulus'] = radius_of_stimulus * variables['copy_times']
    
    # HERE SET SOME VARIABLES FOR THE ELECTRODE CALCULATION
    # distance between masks = 1/6 * lowest dimension
    # radius of electrode = 1/3 * lowest dimension


    ############################################
    ########  Setting up Electrode  ############
    ############################################
    # Adding the coordinates of the electrode to the variables array.
    variables['coord_of_electrode'] = populate_electrode_positions(variables)

    
    # For every run provided in the 'run_id' field of the variables dictionary we perform the simulation (which requires some setting up first).
    for i in range(len(variables['run_id'])):

        ##########################################
        ########  Basic Simulation Setup  ########
        ##########################################
        # Resetting the state of the simulation environment (to avoid interference from the previous run) and seeting the default simulation time step.
        start_scope()
        defaultclock.dt = 0.001*second   

        # Extracting the list of variables required for the current run.
        current_variables = {key: variables[key][i] for key in variables}
        print(current_variables['noise_exc'])

        # Retrieving the treatment settings.
        treatment_settings = [current_variables['device_sensitivity'], current_variables['firing_rate_threshold']]

        # Creating a folder to store the results.
        run_id = current_variables['run_id']
        os.mkdir(f'./results/{run_id}')
        write_run_settings(current_variables, run_id)

        
        #######################################
        ########  Creating Topologies  ########
        #######################################
        # Creating the topologies of both the excitatory and inhibitory neurons.
        layer_data, excitatory_positions, inhibitory_positions = create_complete_neuron_topology_hippocampus_layered(current_variables['N'], current_variables['random_topo_bounds'], current_variables['layer_densities'], current_variables['excitatory_ratios'], current_variables['layer_names'])

        # Assigning the topologies of the excitatory neurons of the different layers to separate variables.
        topology_EC_exc = excitatory_positions['EC']
        topology_DG_exc = excitatory_positions['DG']
        topology_CA1_exc = excitatory_positions['CA1']
        topology_CA3_exc = excitatory_positions['CA3']
        topology_Sub_exc = excitatory_positions['Sub']

        # Assigning the topologies of the inhibitory neurons of the different layers to separate variables.
        topology_EC_inh = inhibitory_positions['EC']
        topology_DG_inh = inhibitory_positions['DG']
        topology_CA1_inh = inhibitory_positions['CA1']
        topology_CA3_inh = inhibitory_positions['CA3']
        topology_Sub_inh = inhibitory_positions['Sub']

        # Adding all the topologies of the neurons to a list.
        topologies_exc = [topology_EC_exc, topology_DG_exc, topology_CA1_exc, topology_CA3_exc, topology_Sub_exc]
        topologies_inh = [topology_EC_inh, topology_DG_inh, topology_CA1_inh, topology_CA3_inh, topology_Sub_inh]

        
        ##########################################
        ########  Creating Stimulus Mask  ########
        ##########################################
        # Creating the stimulus mask. --> MAYBE loop over them all here and simply add all the neurons of a certain layer if it is in the list of layers.
        topology_exc = 0
        for stimulus_layer in stimulus_layers:
            match stimulus_layer:
                case 'EC':
                    topology_exc = topology_EC_exc
                case 'DG':
                    topology_exc = topology_DG_exc
                case 'CA1':
                    topology_exc = topology_CA1_exc
                case 'CA3':
                    topology_exc = topology_CA3_exc
                case 'Sub':
                    topology_exc = topology_Sub_exc

        # Add different implementation for cubical!!!!
        stimulus_mask_exc = create_spherical_mask(topology_exc, stimulus_geometry_settings)

        
        ###########################################
        ########  Creating Treatment Mask  ########
        ###########################################
        # Defining which kind of treatment mask to use.
        treatment_mask = "all_neurons"
        treatment_mask = "all_layer_EC"
        treatment_mask = "spherical_layer_EC"
        
        # Creating the treatment mask where only a sphere of neurons in the middle of the area is included.
        treatment_geometry_settings = [current_variables['coord_of_electrode'], current_variables['radius_of_electrode']]
        treatment_mask_exc = create_spherical_mask(topology_exc, treatment_geometry_settings)
        treatment_mask_inh = create_spherical_mask(topology_inh, treatment_geometry_settings)

        # # Creating the treatment mask where all the neurons are included.
        # treatment_mask_exc = np.ones(current_variables['N'][0])
        # treatment_mask_inh = np.ones(current_variables['N'][1])
        
        treatment_masks = [treatment_mask_exc, treatment_mask_inh]


        ########################################
        ########  Initializing Network  ########
        ########################################
        # Instantiating the network and setting up the monitors.
        net, synapses, monitors = prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables)
        popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors
        
        # Writing the network statistics to a file.
        write_network_statistics(synapses, current_variables['N'], run_id)

        # Plotting and saving the neuron stimulus and treatment mask.
        plot_neuron_masks(topology_exc, [stimulus_mask_exc, treatment_mask_exc], run_id)
        plot_neuron_mask(topology_exc, stimulus_mask_exc, 'red', 'stimulus', run_id)
        plot_neuron_mask(topology_exc, treatment_mask_exc, 'blue', 'treatment', run_id)

        try:
            # Running the simulation with the dynamic objects created above.
            run_granular_simulation(net, current_variables, treatment_settings, monitors)

            # Saving the firing rate data.
            np.savetxt(f'./results/{run_id}/fr_exc.txt', popmon_exc.rate)
            np.savetxt(f'./results/{run_id}/fr_inh.txt', popmon_inh.rate)

            # Plotting the firing rate data together with the noise for both the excitatory and inhibitory neurons.
            plt.plot(popmon_inh.t, popmon_inh.rate, label='Inhibitory')
            plt.plot(popmon_exc.t, popmon_exc.rate, label='Excitatory')

            # Plotting and saving the firing rates.
            plt.legend()
            plt.savefig(f'./results/{run_id}/firing-rates.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the noise for the inhibitory neurons.
            plt.plot(statemon_inh.t, statemon_inh.I_noise[4]/nA, label='inh')
            plt.savefig(f'./results/{run_id}/noise_inh.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the noise for the excitatory neurons.
            plt.plot(statemon_exc.t, statemon_exc.I_noise[4]/nA, label='exc')
            plt.savefig(f'./results/{run_id}/noise_exc.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the recorded LFP voltage values in millivolts from the 'Mlfp' monitor. 
            np.savetxt(f'./results/{run_id}/LFP.txt', Mlfp.v[0]/mV)
            plot(Mlfp.t/ms, Mlfp.v[0]/mV)
            plt.savefig(f'./results/{run_id}/LFP.png', bbox_inches='tight')
            plt.close()

            
        except:
            print('Broken run.')
        
        time.sleep(1)

In [32]:
########################################
########  Simulation Variables  ########
########################################

# Setting the number of times the simulation will be performed.
copy_times = 5

# Defining the layer densities and layer excitatory to inhibitory ratios.
layer_densities_dict = {"EC": 27400, 
                        "DG": 87100, 
                        "CA1": 7900, 
                        "CA3": 15100, 
                        "Sub": 21500}
excitatory_ratios_dict = {"EC": 0.8, 
                          "DG": 0.8, 
                          "CA1": 0.8, 
                          "CA3": 0.8, 
                          "Sub": 0.8}

# Defining the vocabulary of variables.
variables = {
    
    # Defining the simulation settings.
    "run_id": ['Results 1', 'Results 2', 'Results 3', 'Results 4', 'Results 5'],
    "duration": [4000*ms]*copy_times,
    "copy_times": copy_times,

    # Defining the total number of neurons of the model.
    "N": [17000]*copy_times,

    # Defining the bounds used in the random topology model.
    "bounds": [[0.6, 0.6, 0.6]]*copy_times,
    
    # Defining the number of neurons per mm^3 and the excitatory ratio for each of the layers.
    "layer_names": ['EC', 'DG', 'CA1', 'CA3', 'Sub']*copy_times,
    "layer_densities": [dict(layer_densities_dict)]*copy_times,
    "excitatory_ratios": [dict(excitatory_ratios_dict)]*copy_times,

    # Defining the potassium equilibrium potential for both the excitatory and inhibitory neurons.
    # - Healthy mode: Eke_baseline = -90mV
    # - Epileptic mode: Eke_baseline = -84mV
    "Eke_baseline": [-84*mV]*copy_times, 
    "Eki_baseline": [-90*mV]*copy_times,

    # Defining the noise affecting the excitatory and inhibitory neurons.
    "noise_exc": [[0.07, 0.075]*nA]*copy_times, # OLD: [0.1045, 0.104]
    "noise_inh": [[0.05, 0.08]*nA]*copy_times,

    # Defining the base probabilities of connections between neurons:
    # - p_e2e => Probability of an excitatory to excitatory neuron (synapse) connection.
    # - p_e2i => Probability of an excitatory to inhibitory neuron (synapse) connection.
    # - p_i2e => Probability of an inhibitory to excitatory neuron (synapse) connection.
    # - p_i2i => Probability of an inhibitory to inhibitory neuron (synapse) connection.
    # Normal ranges from 0.7-0.75, to activate sprouting increase the normal by 0.5
    # This will increase the average number of excitatory connections by 500.
    "p": [[0.75, 0.35, 0.35, 0.0]]*copy_times, 

    # Defining from which file the stimulus originates.
    "input_signal_file": ['sigmoid-1.0.txt']*copy_times, 

    # Defining the treatment settings.
    "device_sensitivity": [8*ms]*copy_times, # Device sensitivity - how frequently to check is firing rate is above the threshold
    "firing_rate_threshold": [5*Hz]*copy_times, 
    "Eke_treatment": [-100*mV]*copy_times,
    "Eki_treatment": [-90*mV]*copy_times,



    # SHOULD BE UPDATED!!!!
    "radius_of_electrode": [200]*copy_times,
    
    
    
    
    
    "distance_between_masks": [100]*copy_times,
}